## Circle Group

Before we start building circle FFT, let's first build the circle group.

We choose a basic field GF(31), which has a order of 31. And then we extend it to a 2-degree extension field GF(31^2), to get a multiplicative subgroup of order 32.

In [1]:
F31 = GF(31)
R31.<X> = F31[]
R31

Univariate Polynomial Ring in X over Finite Field of size 31

Now we extend F31 to a 2-degree extension field GF(31^2).

In [2]:
C31.<i> = F31.extension(X^2 + 1)
C31

Finite Field in i of size 31^2

We want to find a generator of order 32, so we take the generator of GF(31^2) and get 30th power of it.

**Fact**: For any prime field $\mathbb{F}_p$, where $p=2^n - 1$, the 2-degree extension over $\mathbb{F}_p$ is 
$K=\mathbb{F}_{p^2}$, then $K$ has a multiplicative subgroup with order $2^n$.

Proof:

$$
p^2 - 1 = (2^n - 1) * (2^n - 1) - 1 = 2^{2n} - 2^{n+1} = 2^n(2^n - 2)
$$

In [3]:

g = C31.multiplicative_generator()^(F31.order() - 1)
g

11*i + 29

Define a function to calculate the order of an element in the circle group.

In [4]:
def group_ord(elem):
    K = elem.parent()

    for i in range(1, K.order()):
        if elem**i == K.one():
            return i
    return "False"

Check that g is of order 32.

In [5]:
group_ord(g)

32

Define the group multiplication, which is similar to complex number multiplication.

In [6]:
def group_mul(g1, g2):
    x1, y1 = g1
    x2, y2 = g2
    return (x1 * x2 - y1 * y2) + (x1 * y2 + x2 * y1) * i 

Define the group inverse, which is also the J mapping.

In [7]:
def group_inv(g1):
    x1, y1 = g1
    return (x1 - y1 * i)

Define show function to print group.

In [8]:
def show(GG): 
    for j in range(0, len(GG)):
        t = GG[j]
        t0 = t[0]
        t1 = t[1]
        if t1 > F31(15) and t1 <= F31(30):
            t1_str = "-" + str(31-t1)
        else:
            t1_str = str(t1)
        print("i={}, log={}, ord={}, ({},{})".format(j, log_gen(g, t), group_ord(t), t0, t1_str))

Define log_gen function to calculate the power of an element in the circle group.

In [9]:
def log_gen(gen, t):
    K = gen.parent()
    for i in range(1, K.order()):
        if gen^i == t:
            return i
    return "Fail"

## Jump over the circle

Here we define the generator for `Circle Group` as chosen by Vitalik https://vitalik.eth.limo/general/2024/07/23/circlestarks.html

In [12]:
# optional
g = 10 + 5*i
group_ord(g)

32

In [13]:
g * g

7*i + 13

In [14]:
G5 = [g^i for i in range(0, 2^5)]
# G5's elements must be on the circle x^2 + y^2 = 1
# due to:
#   a^(p + 1) == 1, a = x + y * i
#   a * a ^ p == (x + y * i) * (x - y * i) == x^2 + y^2 == 1
for t in G5:
    x, y = t
    assert x^2 + y^2 == 1
G5

[1,
 5*i + 10,
 7*i + 13,
 11*i + 2,
 27*i + 27,
 2*i + 11,
 13*i + 7,
 10*i + 5,
 i,
 10*i + 26,
 13*i + 24,
 2*i + 20,
 27*i + 4,
 11*i + 29,
 7*i + 18,
 5*i + 21,
 30,
 26*i + 21,
 24*i + 18,
 20*i + 29,
 4*i + 4,
 29*i + 20,
 18*i + 24,
 21*i + 26,
 30*i,
 21*i + 5,
 18*i + 7,
 29*i + 11,
 4*i + 27,
 20*i + 2,
 24*i + 13,
 26*i + 10]

In [15]:
for j in range(0, 32):
    t = g^j
    print("i={}, log={}, ord={}, ({},{})".format(j, log_gen(g, t), group_ord(t), t[0], t[1]))

i=0, log=32, ord=1, (1,0)
i=1, log=1, ord=32, (10,5)
i=2, log=2, ord=16, (13,7)
i=3, log=3, ord=32, (2,11)
i=4, log=4, ord=8, (27,27)
i=5, log=5, ord=32, (11,2)
i=6, log=6, ord=16, (7,13)
i=7, log=7, ord=32, (5,10)
i=8, log=8, ord=4, (0,1)
i=9, log=9, ord=32, (26,10)
i=10, log=10, ord=16, (24,13)
i=11, log=11, ord=32, (20,2)
i=12, log=12, ord=8, (4,27)
i=13, log=13, ord=32, (29,11)
i=14, log=14, ord=16, (18,7)
i=15, log=15, ord=32, (21,5)
i=16, log=16, ord=2, (30,0)
i=17, log=17, ord=32, (21,26)
i=18, log=18, ord=16, (18,24)
i=19, log=19, ord=32, (29,20)
i=20, log=20, ord=8, (4,4)
i=21, log=21, ord=32, (20,29)
i=22, log=22, ord=16, (24,18)
i=23, log=23, ord=32, (26,21)
i=24, log=24, ord=4, (0,30)
i=25, log=25, ord=32, (5,21)
i=26, log=26, ord=16, (7,18)
i=27, log=27, ord=32, (11,29)
i=28, log=28, ord=8, (27,4)
i=29, log=29, ord=32, (2,20)
i=30, log=30, ord=16, (13,24)
i=31, log=31, ord=32, (10,26)


In [16]:
G5[1] * G5[1] == G5[2]

True

In [17]:
G5[3] * G5[4] == G5[7]

True

In [18]:
x1, y1 = G5[1]
x1, y1

(10, 5)

In [19]:
group_mul(G5[1], G5[2]) == G5[3]

True

In [20]:
group_inv(G5[1]) ==  G5[31], G5[1] * group_inv(G5[1])

(True, 1)

In [21]:
G4 = [t^2 for t in G5[:16]]
G4

[1,
 7*i + 13,
 27*i + 27,
 13*i + 7,
 i,
 13*i + 24,
 27*i + 4,
 7*i + 18,
 30,
 24*i + 18,
 4*i + 4,
 18*i + 24,
 30*i,
 18*i + 7,
 4*i + 27,
 24*i + 13]

In [22]:
G3 = [t^2 for t in G4[:8]]
G3

[1, 27*i + 27, i, 27*i + 4, 30, 4*i + 4, 30*i, 4*i + 27]

## Standard Position Coset

$D$ is a standard position coset of size $2^n$

$$
D = Q\cdot G_n
$$

where $ord(Q)=2^{n+1}$.

And $D$ is also a twin-coset of size $2^n$ defined with $G_{n-1}$:

$$
D = Q\cdot G_{n-1} \uplus Q^{-1}\cdot G_{n-1}
$$

The rotation $Q^2$ is exactly the generator of $G_n$.

In [23]:
# Define a coset or G4 from a rotation Q, ord(Q) = 32
#
# Let Q = g (generator of G, whose order is 32)

D_standard_coset_4 = [g * t for t in G4]; show(D_standard_coset_4)

i=0, log=1, ord=32, (10,5)
i=1, log=3, ord=32, (2,11)
i=2, log=5, ord=32, (11,2)
i=3, log=7, ord=32, (5,10)
i=4, log=9, ord=32, (26,10)
i=5, log=11, ord=32, (20,2)
i=6, log=13, ord=32, (29,11)
i=7, log=15, ord=32, (21,5)
i=8, log=17, ord=32, (21,-5)
i=9, log=19, ord=32, (29,-11)
i=10, log=21, ord=32, (20,-2)
i=11, log=23, ord=32, (26,-10)
i=12, log=25, ord=32, (5,-10)
i=13, log=27, ord=32, (11,-2)
i=14, log=29, ord=32, (2,-11)
i=15, log=31, ord=32, (10,-5)


In [24]:
# D_twin_coset_4 is equal to D_standard_coset_4 in the set-theoretic sense, with different order

D_twin_coset_4 = [g * t for t in G3] + [group_inv(g) * t for t in G3]; show(D_standard_coset_4)

# `log` specifies the order of an element in the circle

i=0, log=1, ord=32, (10,5)
i=1, log=3, ord=32, (2,11)
i=2, log=5, ord=32, (11,2)
i=3, log=7, ord=32, (5,10)
i=4, log=9, ord=32, (26,10)
i=5, log=11, ord=32, (20,2)
i=6, log=13, ord=32, (29,11)
i=7, log=15, ord=32, (21,5)
i=8, log=17, ord=32, (21,-5)
i=9, log=19, ord=32, (29,-11)
i=10, log=21, ord=32, (20,-2)
i=11, log=23, ord=32, (26,-10)
i=12, log=25, ord=32, (5,-10)
i=13, log=27, ord=32, (11,-2)
i=14, log=29, ord=32, (2,-11)
i=15, log=31, ord=32, (10,-5)


In [25]:
D_twin_coset_4_1 = [G5[3] * t for t in G3] + [group_inv(G5[3]) * t for t in G3]; show(D_twin_coset_4_1)

i=0, log=3, ord=32, (2,11)
i=1, log=7, ord=32, (5,10)
i=2, log=11, ord=32, (20,2)
i=3, log=15, ord=32, (21,5)
i=4, log=19, ord=32, (29,-11)
i=5, log=23, ord=32, (26,-10)
i=6, log=27, ord=32, (11,-2)
i=7, log=31, ord=32, (10,-5)
i=8, log=29, ord=32, (2,-11)
i=9, log=1, ord=32, (10,5)
i=10, log=5, ord=32, (11,2)
i=11, log=9, ord=32, (26,10)
i=12, log=13, ord=32, (29,11)
i=13, log=17, ord=32, (21,-5)
i=14, log=21, ord=32, (20,-2)
i=15, log=25, ord=32, (5,-10)


In [26]:
D_twin_coset_4_2 = [G5[5] * t for t in G3] + [group_inv(G5[5]) * t for t in G3]; show(D_twin_coset_4_2)

i=0, log=5, ord=32, (11,2)
i=1, log=9, ord=32, (26,10)
i=2, log=13, ord=32, (29,11)
i=3, log=17, ord=32, (21,-5)
i=4, log=21, ord=32, (20,-2)
i=5, log=25, ord=32, (5,-10)
i=6, log=29, ord=32, (2,-11)
i=7, log=1, ord=32, (10,5)
i=8, log=27, ord=32, (11,-2)
i=9, log=31, ord=32, (10,-5)
i=10, log=3, ord=32, (2,11)
i=11, log=7, ord=32, (5,10)
i=12, log=11, ord=32, (20,2)
i=13, log=15, ord=32, (21,5)
i=14, log=19, ord=32, (29,-11)
i=15, log=23, ord=32, (26,-10)


In [27]:
# Fact:
#   g^2 is the generator of G4
#   g * G4 is the standard position coset

G4 == [(g^2)^i for i in range(0, 16)]

True

In [28]:
# Define a coset or G3 from a rotation Q, ord(Q) = 16
#
# Let Q = g^2 (g is the generator of G5, whose order is 32)

D_standard_coset_3 = [g^2 * t for t in G3]; show(D_standard_coset_3)

i=0, log=2, ord=16, (13,7)
i=1, log=6, ord=16, (7,13)
i=2, log=10, ord=16, (24,13)
i=3, log=14, ord=16, (18,7)
i=4, log=18, ord=16, (18,-7)
i=5, log=22, ord=16, (24,-13)
i=6, log=26, ord=16, (7,-13)
i=7, log=30, ord=16, (13,-7)


## Squaring map

**Lemma**: For any twin-coset $D$ of size $N=2^n$, the image of $\pi(D)$ must be a twin-coset of size $2^{n-1}$.

$$
D = Q\cdot G_{n-1} \uplus Q^{-1}\cdot G_{n-1}
$$

$$
\pi(D) = \pi(Q)\cdot G_{n-2} \uplus \pi(Q)^{-1}\cdot G_{n-2}
$$

**Fact**: If $D$ is a standard position coset, so is $\pi(D)$.

In [29]:
def sq(D):
    rs = []
    for t in D:
        if t^2 not in rs:
            rs += [t^2]  
    return rs

In [30]:
D_standard_coset_3 = sq(D_standard_coset_4); show(D_standard_coset_3)

i=0, log=2, ord=16, (13,7)
i=1, log=6, ord=16, (7,13)
i=2, log=10, ord=16, (24,13)
i=3, log=14, ord=16, (18,7)
i=4, log=18, ord=16, (18,-7)
i=5, log=22, ord=16, (24,-13)
i=6, log=26, ord=16, (7,-13)
i=7, log=30, ord=16, (13,-7)


## Vanishing Polynomial

In [31]:
v = [x]
for idx in range(1, 5):
    v.append(2 * v[idx-1] ^ 2 - 1)

v

[10, 13, 27, 0, 30]

## Circle FFT

In [32]:
f = {}
from random import randint

for idx, t in enumerate(D_standard_coset_4):
    f[t] = C31(randint(0, 100))

f

{5*i + 10: 3,
 11*i + 2: 28,
 2*i + 11: 15,
 10*i + 5: 7,
 10*i + 26: 29,
 2*i + 20: 0,
 11*i + 29: 2,
 5*i + 21: 5,
 26*i + 21: 20,
 20*i + 29: 15,
 29*i + 20: 0,
 21*i + 26: 6,
 21*i + 5: 12,
 29*i + 11: 16,
 20*i + 2: 6,
 26*i + 10: 2}

In [33]:
def pi_value(t):
    # x^2 - y^2 == 2 * x^2 - 1 (x^2 + y^2 = 1)
    return C31(2 * t^2 - 1)

In [34]:
def ifft_first_step(f):
    f0 = {}
    f1 = {}
    for t in f:
        x, y = t

        f0[x] = (f[t] + f[group_inv(t)]) / C31(2)
        f1[x] = (f[t] - f[group_inv(t)]) / (C31(2) * y)

        assert f[t] == f0[x] + y * f1[x]

        # print("{}: {} = {} + {} * {}".format(t, f[t], f0[x], f1[x], y))

    return f0, f1

In [35]:
def ifft_normal_step(f, debug_output=None):
    if debug_output:
        debug_output.append(f)

    if len(f) == 1:
        res = []
        for x in f:
            res.append(f[x])
        return res

    f0 = {}
    f1 = {}

    for x in f:
        assert x != 0, "f should be on coset"
        f0[pi_value(x)] = (f[x] + f[-x]) / C31(2)
        f1[pi_value(x)] = (f[x] - f[-x]) / (C31(2) * x)
        # print('f[{}] - f[{}] = {} - {}'.format(x, -x, f[x], f[-x]))

        assert f[x] == f0[pi_value(x)] + x * f1[pi_value(x)]

        # print("x={}: {} = {} + {} * {}".format(x, f[x], f0[pi(x)], f1[pi(x)], x))
        # print("pi(x)={}".format(pi(x)))

    return ifft_normal_step(f0, debug_output=debug_output) + ifft_normal_step(f1, debug_output=debug_output)

In [36]:
def ifft(f, debug_output=None):
    f0, f1 = ifft_first_step(f)
    if debug_output:
        debug_output.append(f0)
        debug_output.append(f1)
    f0 = ifft_normal_step(f0, debug_output=debug_output)
    f1 = ifft_normal_step(f1, debug_output=debug_output)

    return f0 + f1

In [37]:
coeffs = ifft(f)
coeffs

[22, 30, 1, 17, 19, 27, 2, 14, 24, 7, 30, 10, 22, 20, 26, 4]

In [38]:
def pie_group(D):
    D_new = []
    for x in D:
        x_new = C31(2 * x^2 - 1)
        if x_new not in D_new:
            D_new.append(x_new)

    assert len(D_new) * 2 == len(D), "len(D_new) * 2 != len(D), {} * 2 != {}, D_new={}, D={}".format(len(D_new), len(D), D_new, D)
    
    return D_new

In [39]:
def fft_first_step(f, D):
    assert len(f) == len(D), "len(f) != len(D), {} != {}, f={}, D={}".format(len(f), len(D), f, D)

    len_f = len(f)
    f0 = f[:len_f//2]
    f1 = f[len_f//2:]

    D_new = []
    for t in D:
        x, _ = t
        if x not in D_new:
            D_new.append(x)

    assert len(D_new) * 2 == len(D), "len(D_new) * 2 != len(D), {} * 2 != {}, D_new={}, D={}".format(len(D_new), len(D), D_new, D)

    return f0, f1, D_new

In [40]:
def fft_normal_step(f, D, debug_output=None):
    if debug_output:
        debug_output.append(f)

    if len(f) == 1:
        return {D[0]: f[0]}
    
    next_domain = pie_group(D)
    assert len(next_domain) * 2 == len(D), "len(next_domain) * 2 != len(D), {} * 2 != {}, next_domain={}, D={}".format(len(next_domain), len(D), next_domain, D)
    assert len(f) == len(D), "len(f) != len(D), {} != {}, f={}, D={}".format(len(f), len(D), f, D)
    f0 = fft_normal_step(f[:len(f)//2], next_domain, debug_output)
    f1 = fft_normal_step(f[len(f)//2:], next_domain, debug_output)

    f_new = {}
    for x in D:
        f_new[x] = f0[pi_value(x)] + f1[pi_value(x)] * x

    for x in D:
        if x != 0:
            assert f0[pi_value(x)] == (f_new[x] + f_new[-x]) / C31(2), "f0[pi(x)] = {}".format(f0[pi_value(x)])
            assert f1[pi_value(x)] == (f_new[x] - f_new[-x]) / (C31(2) * x), "f1[pi(x)] = {}".format(f1[pi_value(x)])
        else:
            assert f0[pi_value(x)] == f_new[x], "f0[pi(x)] = {}".format(f0[pi_value(x)])

    assert len(f) == len(f_new), "len(f) != len(f_new), {} != {}, f={}, f_new={}, D={}".format(len(f), len(f_new), f, f_new, D)
    assert ifft_normal_step(f_new) == f, "ifft(f_new) != f, {} != {}".format(ifft_normal_step(f_new), f)
        
    return f_new

In [41]:
def fft(f, D, debug_output=None):

    assert len(f) == len(D), "len(f) != len(D), {} != {}, f={}, D={}".format(len(f), len(D), f, D)

    D_copy = D[:]
    f0, f1, D = fft_first_step(f, D)

    assert len(f0) == len(D), "len(f0) != len(D), {} != {}, f0={}, D={}".format(len(f0), len(D), f0, D)
    assert len(f1) == len(D), "len(f1) != len(D), {} != {}, f1={}, D={}".format(len(f1), len(D), f1, D)

    if debug_output:
        debug_output.append(f0)
        debug_output.append(f1)

    f0 = fft_normal_step(f0, D, debug_output)
    f1 = fft_normal_step(f1, D, debug_output)

    f = {}
    for t in D_copy:
        x, y = t
        f[t] = f0[x] + f1[x] * y

    return f


In [42]:
f_prime = fft(coeffs, D_standard_coset_4)

for t, s in zip(f, f_prime):
    assert t == s

for t in f:
    print("f[{}]={}, f_prime[{}]={}".format(t, f[t], t, f_prime[t]))
    assert f[t] == f_prime[t], "f[{}] != f_prime[{}], {} != {}".format(t, t, f[t], f_prime[t])

f[5*i + 10]=3, f_prime[5*i + 10]=3
f[11*i + 2]=28, f_prime[11*i + 2]=28
f[2*i + 11]=15, f_prime[2*i + 11]=15
f[10*i + 5]=7, f_prime[10*i + 5]=7
f[10*i + 26]=29, f_prime[10*i + 26]=29
f[2*i + 20]=0, f_prime[2*i + 20]=0
f[11*i + 29]=2, f_prime[11*i + 29]=2
f[5*i + 21]=5, f_prime[5*i + 21]=5
f[26*i + 21]=20, f_prime[26*i + 21]=20
f[20*i + 29]=15, f_prime[20*i + 29]=15
f[29*i + 20]=0, f_prime[29*i + 20]=0
f[21*i + 26]=6, f_prime[21*i + 26]=6
f[21*i + 5]=12, f_prime[21*i + 5]=12
f[29*i + 11]=16, f_prime[29*i + 11]=16
f[20*i + 2]=6, f_prime[20*i + 2]=6
f[26*i + 10]=2, f_prime[26*i + 10]=2
